Topic Modelling

In [42]:
import spacy
import re
import spacy_transformers
import spacy_cleaner
from spacy_cleaner.processing import removers, replacers, mutators
import torch
import json
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
nlp = spacy.load('de_dep_news_trf')

In [26]:
pipeline = spacy_cleaner.Pipeline(
    nlp,
    removers.remove_stopword_token,
    replacers.replace_punctuation_token,
    mutators.mutate_lemma_token,
)

In [69]:
with open('BundestagsReden.json') as json_file:
    data = json.load(json_file)

Lowercase is already done during forming dictionary

In [70]:
print(data['ID20100600'][1])

sehr geehrter herr alterspräsident dr. schäuble! also, herr brandner, gleich in der ersten sitzung die braune widerlichkeitsskala in solche höhen zu treiben,das ist immerhin respektvoll.dass ausgerechnet sie willy brandt zitieren, der gegen die nazis gekämpft hat, der niedergekniet ist vor den opfern des aufstandes im warschauer ghetto, dass ausgerechnet sie, die in der tradition der nazis stehen,das hier anbringen, ist abartig, um das in aller klarheit zu sagen. abartig und ekelerregend!liebe kolleginnen und kollegen, sehr geehrte damen und herren von der afd,wenn ich mir die politische lage angucke, dann finde ich zurzeit besonders wichtig: wie entwickelt sich corona? ich finde es wichtig, darüber zu diskutieren, wie wir kinderarmut bekämpfen können. und ich mache mir große sorgen, dass sich hier eine koalition anbahnt, in der das soziale hinten runterfallen wird, weil die stabil marktradikale fdp dafür schon sorgen wird.das ist etwas, was mich und viele menschen bewegt.was ist der a

In [15]:
doc = nlp(data['ID20100600'][1].replace('–', ''))

In [16]:
# doc = nlp(data)

Tokenize

In [29]:
for token in doc:
    if (not token.is_punct 
    and not token.is_currency
    and not token.is_digit
    and not token.is_space
    and not token.is_stop
    and not token.like_num):
        print(token.lemma_)

geehrt
Herr
Alterspräsident
dr
Schäuble
Herr
Brandner
Sitzung
braun
Widerlichkeitsskala
Höhe
treiben
immerhin
respektvoll.dass
ausgerechnet
Willy
Brandt
zitieren
Nazi
kämpfen
niederknieen
Opfer
Aufstand
Warschauer
Ghetto
ausgerechnet
Tradition
Nazi
stehen
anbringen
abartig
Klarheit
sagen
abartig
ekelerregend
lieb
Kollegin
Kollege
geehrt
Dame
Herren
afd
politisch
Lage
angucken
finden
zurzeit
wichtig
entwickeln
Corona
finden
wichtig
diskutieren
Kinderarmut
bekämpfen
machen
Sorge
Koalition
anbahnen
sozial
hinten
runterfallen
stabil
marktradikal
FDP
sorgen
wird.das
Mensch
bewegt.was
afd
wichtig
afd
wichtig
Verunglimpfung
geschlechtergerecht
Sprache
einzig
sinnvoll
Änderungsantrag
sehen
parlamentarisch
verblödet
eigentlich
mal
sagen
Änderungsantrag
einzig
erkennen
offenbar
Bundestag
Gesetzestext
lesen
binnen-i
doppelpunkte
gendersternchen.das
absurd
beantragen
Gesetzestext
völlig
Papierkorb
vorlegen
Fraktion
Vorlage
sozialdemokratisch
Fraktion
unterstützen
halten
Geschäftsordnung
Grundlage


In [30]:
for token in doc:
    print(token.lemma_)

sehr
geehrt
Herr
Alterspräsident
dr
.
Schäuble
--
also
--
Herr
Brandner
--
gleich
in
der
erster
Sitzung
der
braun
Widerlichkeitsskala
in
solcher
Höhe
zu
treiben
--
der
sein
immerhin
respektvoll.dass
ausgerechnet
sie
Willy
Brandt
zitieren
--
der
gegen
der
Nazi
kämpfen
haben
--
der
niederknieen
sein
vor
der
Opfer
der
Aufstand
in
Warschauer
Ghetto
--
dass
ausgerechnet
sie
--
der
in
der
Tradition
der
Nazi
stehen
--
der
hier
anbringen
--
sein
abartig
--
um
der
in
aller
Klarheit
zu
sagen
--
abartig
und
ekelerregend
--
lieb
Kollegin
und
Kollege
--
sehr
geehrt
Dame
und
Herren
von
der
afd
--
wenn
ich
sich
der
politisch
Lage
angucken
--
dann
finden
ich
zurzeit
besonders
wichtig
--
wie
entwickeln
sich
Corona
--
ich
finden
es
wichtig
--
darüber
zu
diskutieren
--
wie
wir
Kinderarmut
bekämpfen
können
--
und
ich
machen
mir
groß
Sorge
--
dass
sich
hier
ein
Koalition
anbahnen
--
in
der
der
sozial
hinten
runterfallen
werden
--
weil
der
stabil
marktradikal
FDP
dafür
schon
sorgen
wird.das
sein
etwas
--
we

def preprocessing (textToProcess):
    #tokenize
    #remove special characters
    #remove stopwords
    #lemmata
    processedText = pipeline.clean(textToProcess)
    return processedText

In [71]:
#iterate over all texts
alltexts = []
tokenized_corpus = []
for key in data:
    #alltexts += data[key][1]
    alltexts.append(data[key][1])
    #print(data[key][1])
    #doc = nlp(data[key][1].replace('–', ''))
    #processedText = preprocessing(data[key][1].replace('–', ''))
    #print(preprocessing(data[key][1].replace('–', '')))
    #print(processedText)

In [68]:
print(alltexts)

['sehr geehrter herr alterspräsident! so muss ich es jetzt sagen, genau; ich muss mich daran gewöhnen.nicht lange.\n\t\t\t\t\tnicht lange; da haben sie schon recht. – liebe kolleginnen und kollegen! meine sehr geehrten damen und herren! die übernahme der geschäftsordnung ist eine der ersten wichtigen entscheidungen, die wir heute zu beginn der wahlperiode treffen müssen. die geschäftsordnung ist grundlage für unsere gemeinsame arbeit. sie hat sich über viele wahlperioden bewährt und wird uns auch durch die neue wahlperiode tragen. es ist gute tradition, dass wir die geschäftsordnung am anfang einer wahlperiode mit breiter parlamentarischer mehrheit übernehmen, und das sollten wir auch heute tun.sie ist nicht in stein gemeißelt – das wissen diejenigen, die bereits in der letzten wahlperiode hier im bundestag tätig waren –, sondern die geschäftsordnung ist immer ein, ich sage mal, lebendiges dokument, das sich den gegebenheiten des parlamentarismus, aber auch den gegebenheiten der arbeit

NameError: name 'path_to_files' is not defined

In [72]:
tokenized_corpus = []
for texts in alltexts:
    doc = nlp(texts)
    tokenized_texts = []
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.like_num and token.lemma_.lower() not in extra_stop:
            tokenized_texts.append(token.lemma_.lower())
    
    tokenized_corpus.append(tokenized_texts)

In [62]:
#print(tokenized_corpus)

[['geehrt', 'herr', 'alterspräsident', 'sagen', 'genau', 'gewöhnen.nicht', '\n\t\t\t\t\t', 'lieb', 'kollegin', 'kollege', 'geehrt', 'dame', 'herr', 'übernahme', 'geschäftsordnung', 'wichtig', 'entscheidung', 'beginn', 'wahlperiode', 'treffen', 'geschäftsordnung', 'grundlage', 'gemeinsam', 'arbeit', 'wahlperiode', 'bewähren', 'wahlperiode', 'tragen', 'tradition', 'geschäftsordnung', 'anfang', 'wahlperiode', 'breit', 'parlamentarisch', 'mehrheit', 'übernehmen', 'tun.sie', 'stein', 'meißeln', 'wissen', 'letzter', 'wahlperiode', 'bundestag', 'tätig', 'geschäftsordnung', 'sagen', 'mal', 'lebendig', 'dokument', 'gegebenheit', 'parlamentarismus', 'gegebenheit', 'arbeit', 'deutsch', 'bundestag', 'anpassen', 'geschäftsordnung', 'entwicklung', 'nachvollziehen', 'letzter', 'wahlperiode', 'beweisen', 'grundlegend', 'änderung', 'beschließen', 'voranstellen', 'überfällig', 'reform', 'verhaltensregeln.wir', 'begrüßen', 'ausdrücklich', 'initiative', 'scheidend', 'präsident', 'herrn', 'schäuble', 'gesc

In [73]:
words_id = corpora.Dictionary(tokenized_corpus)
corpus = [words_id.doc2bow(txt) for txt in tokenized_corpus]

In [74]:
k_init = 5
k_final = 15

In [75]:
for k in range(k_init,k_final+1):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=words_id, num_topics = k, random_state=50, passes=20, per_word_topics=True)

    per_lda = lda_model.log_perplexity(corpus)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=words_id, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(k,per_lda,coherence_lda)

5 -8.747976360237809 0.39125068808572183
6 -8.799079918428276 0.4580486298411026
7 -8.890935174262296 0.4245892898568475
8 -9.025048216630331 0.3992237579890396
9 -9.16321243091898 0.4470115698033737
10 -9.302410507053294 0.40715096452821475
11 -9.416761355662922 0.4497611407376368
12 -9.534755215093146 0.47059382203456585
13 -9.635296921402091 0.46582945425237743
14 -9.715957934398075 0.49952290070175315
15 -9.82675837688745 0.49452832701074406


In [76]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=words_id, num_topics = 14, random_state=50, passes=20, per_word_topics=True)

In [77]:
lda_model.show_topics(num_words=20, num_topics=14)

[(0,
  '0.018*"impfpflicht" + 0.016*"pandemie" + 0.013*"impfung" + 0.013*"mensch" + 0.011*"pflege" + 0.011*"impfen" + 0.010*"lauterbach" + 0.009*"patient" + 0.009*"maßnahme" + 0.009*"gesundheitssystem" + 0.008*"impfstoff" + 0.008*"medizinisch" + 0.007*"gesundheitswesen" + 0.007*"gesundheit" + 0.006*"schwer" + 0.006*"versorgung" + 0.006*"schützen" + 0.005*"pflegekräfte" + 0.005*"medikament" + 0.005*"krankheit"'),
 (1,
  '0.015*"mensch" + 0.013*"frau" + 0.013*"kind" + 0.009*"wichtig" + 0.008*"brauchen" + 0.008*"kollegin" + 0.007*"bildung" + 0.007*"jung" + 0.007*"land" + 0.006*"geehrt" + 0.006*"million" + 0.006*"familie" + 0.005*"leben" + 0.005*"lieb" + 0.005*"unser" + 0.005*"landwirtschaft" + 0.005*"schule" + 0.005*"kollege" + 0.005*"deutschland" + 0.004*"nachhaltig"'),
 (2,
  '0.011*"bundesverfassungsgericht" + 0.010*"petition" + 0.009*"rechtsstaat" + 0.008*"justiz" + 0.008*"gericht" + 0.008*"urteil" + 0.007*"219a" + 0.007*"strafrecht" + 0.007*"betroffener" + 0.007*"strafgesetzbuch" + 0

In [82]:
lda_model[corpus[1]]

([(10, 0.99529874)],
 [(5, [10]),
  (10, [10]),
  (16, [10]),
  (19, [10]),
  (22, [10]),
  (23, [10]),
  (25, [10]),
  (38, [10]),
  (42, [10]),
  (44, [10]),
  (45, [10]),
  (48, [10]),
  (49, [10]),
  (54, [10]),
  (55, [10]),
  (60, [10]),
  (61, [10]),
  (62, [10]),
  (63, [10]),
  (69, [10]),
  (71, [10]),
  (73, [10]),
  (76, [10]),
  (79, [10]),
  (80, [10]),
  (82, [10]),
  (89, [10]),
  (90, [10]),
  (96, [10]),
  (97, [10]),
  (102, [10]),
  (103, [10]),
  (111, [10]),
  (112, [10]),
  (113, []),
  (114, [10]),
  (115, [10]),
  (116, []),
  (117, [10]),
  (118, [10]),
  (119, [10]),
  (120, [10]),
  (121, [10]),
  (122, [10]),
  (123, []),
  (124, [10]),
  (125, [10]),
  (126, [10]),
  (127, []),
  (128, [10]),
  (129, [10]),
  (130, [10]),
  (131, [10]),
  (132, [10]),
  (133, [10]),
  (134, [10]),
  (135, [10]),
  (136, [10]),
  (137, [10]),
  (138, [10]),
  (139, [10]),
  (140, [10]),
  (141, []),
  (142, [10]),
  (143, [10]),
  (144, [10]),
  (145, [10]),
  (146, [10]),
